# Problem
We would like to calibrate a GridLAB-D house model based on a data collected from a house.  The house parameters we wish to calibrate are the indoor-outdoor and air-mass conductivities $U_A$ and $U_M$, the air and mass heat capacities $C_A$ and $C_M$, and the HVAC system heating and cooling efficieicies $\eta_H$ and $\eta_C$.  The available data is the hourly outdoor air temperature $T_O$, indoor air temperature $T_A$, heating and cooling setpoints $T_H$ and $T_C$, and the total electric energy demand of the HVAC system.

# Methodology
To solve this problem we must perform a piecewise regression based on a second order thermal model of the house.  The thermal model we will calibrate is given by two differential equations for the heat balance:
$$
    \left \{ \begin{array}{lclclcl}
        U_A(T_A-T_O) &-& U_M(T_A-T_M) &+& C_A \dot T_A & = & Q
    \\
        & & U_M(T_A-T_M) &+& C_M \dot T_M & = & 0.
    \end{array} \right .
$$
where the heat gain $Q = Q_i + Q_h + Q_c$.  We will evaluate $Q_i = \eta_i ~ P_i(t)$, $Q_h = \eta_h ~ P_h(t)$, and $Q_c = \eta_c ~ P_c(t)$
with
* $P_i$ is the power supplied to the enduses,
* $P_h$ is the power supplied to the heating system,
* $P_c$ is the power supplied to the cooling system,
* $\eta_i$ is the fraction of internal heat gains 
* $\eta_h$ is the heating system coefficient of performance, and
* $\eta_c$ is the cooling system coefficient of performance.

We combine the two equations to find the total house heat balance:
$$
    U_A(T_A-T_O) + C_A \dot T_A + C_M \dot T_M = Q.
$$

When the indoor air temperature is maintained at the constant setpoint $T_S$ we have $\int_t^{t+\Delta t} T_A(t) ~ dt = T_S ~ \Delta t$ and $\dot T_a = 0 \implies \dot T_M = 0$. Then we can integrate $Q$ over the interval $(t,t+\Delta t)$ to find
$$
    E(t,\Delta t) = U_A ~ T(t,\Delta t).
$$
where the value of $D_H$ is determined based on the setpoint temperature, e.g.,
$$
    T(t) = \left\{ \begin{array}{lll}
        T_O(t) < T_H & : & \int_t^{t+\Delta t} [ T_O(t) - T_H ] ~ dt
    \\
        T_O(t) > T_C & : & \int_t^{t+\Delta t} [ T_O(t) - T_C ] ~ dt
    \\
        otherwise & : & 0
    \end{array} \right.
$$

We can estimate the value of $U_A$ by computing
$$
    U_A = (\mathbf{A}^T~\mathbf{A})^{-1} ~ \mathbf{A}^T ~ \mathbf{y}
$$
where
$$
    \begin{array}{cc}
        \mathbf{A} = \begin{bmatrix}
            D_H(0) \\
            \vdots \\
            D_H(8760)
        \end{bmatrix} &
        \mathrm{and} &
        \mathbf{y} = \begin{bmatrix}
            E(0) \\
            \vdots \\
            E(8760)
        \end{bmatrix}
    \end{array}
$$

# Solution
We generate a dataset using the GridLAB-D model in [house.glm](http://localhost:8888/edit/house.glm). This model is design to be run twice.  The first time it runs a reference model, and the second time it runs the calibrated model.  At the end of the first run, the model calibration process is run (see [calibrate.py](http://localhost:8888/edit/calibrate.py)) and the second time, the model evaluation process is run (see [evaluate.py](http://localhost:8888/edit/evaluate.py)).

In [10]:
!gridlabd house.glm

WARNING  [INIT] : house_e:1 has no parent triplex_meter defined; using static voltages
Processing 2010-12-26 15:00:00 PST...